In [ ]:
from pathlib import Path
import pandas as pd
from src.data import load_data, filter_notes, filter_patients

data_dir = Path("/gpfs/data/benjamin-lab/Trauma_LLM/data")
window = 24 # hours

In [ ]:
data = load_data(data_dir)
patients = filter_patients(
    registry=data["registry"],
    demo=data["demo"],
    disch=data["disch"],
)
notes = filter_notes(
    patients=patients,
    window=window,
    note_types=["hp", "op", "tert"],
    **data,
)

In [ ]:
notes.groupby("SOURCE")["NOTE_ID"].nunique()

In [ ]:
notes.groupby("SOURCE")["MRN"].nunique()

In [ ]:
# low number of tertiary notes from source, compared to total number of records in registry
print("Unique patients with tertiary note", data["tert"]["MRN"].nunique())
print("Unique patients in registry", data["registry"]["Medical Record #"].nunique())

In [ ]:
notes["MRN"].nunique()